In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'learning-agency-lab-automated-essay-scoring-2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F71485%2F8059942%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240617%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240617T132855Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9ee13f33726e2e8a37b5d5088475be5e7d873f2b9b254923636929d25818fdba027c9152da82ec5861906e149a154f6eed948d758f278ad94b13076a50f3bc29b531c834ca2b9feed42ccd79d30e21c6bfc1c77ba645b119ba2e7ec00e1f2c52168fa440a72035e3ac4b6c5f40d7415224cfda416dcecb62fae0fec9d7520f05a531fd2ae4e383fc268feda986d0bdced088c4143aad2ce521a0c7b0fd0330e1e30adad27a80579a2c7f515017faa0d300f6689d0533b496eacf37dbdbc1352863bd394a72f8fd9455ab39a7984346406b837c0c0e94aab98ad188a51fe5e3479161d6c86ed0fec5fb58b77a158e9a5dde2ada3978823331825239cc85dae08c,huggingfacedebertav3variants:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2663421%2F4620664%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240617%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240617T132856Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db0c1b8f8bd99f530abe801115018f69a3561bcf5ddaf0721900c4998bbf04dfbbb1c55c902ad0b0c6fc7987e1fe5e4c4125600bc6e968d701ac6e4f99582fddea3c0559c777d5aa12a61ad0f248e530292bc1c28839791edc85c3851293b8adfc59b4df12a037a75379e659f2cd5c2ecb771626306749c31aab7f88590a6e7bc1f89806136251772f6921db6f731c342ba56628363d4e160bec39c3fae47c8e7e7575625c88b02e5b884a78fa4d009ea3352c917ac85d883c41aa490de3d6dacccac17ab98c7a2c0530f31daf1fb77a4fdb603bd723782937b38ddcab2523a40e801a34e514530f41808d0235773b749e1e30e28382f162938a7db7eda5f013e,mistral/pytorch/7b-v0.1-hf/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F3899%2F5111%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240617%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240617T132856Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3fb94dbf069d9b6a253101a1ab95f15ad2e704850569a14fad781ffbd3049ecfcc1c44cfd6a88b1c5a0cfa1959c2d4fbcd3bd858f09ba7174e64577d19487695a1a2f0ca0dd6f263a4268847053d47697b69defec9ee13f858f2b544df77974c90a3087e1c96fd9a5cc0118650541c3d961407d5ff5dcec3114d63e8df4df9b753d2c0f6806efdf845cce6a91c30285ff899aa3dacdf0d0ea26e496cb08d63148ed4669fcf03b417d273003e84e76e7a4718ac64de03e7d688de3d5a5d60e59511f4b0f9d10eb1aeb5277cb21ea57bf51b143ba14af7ae49c7a7f8c4e188a1cbbb10fd8f340435bed665a11e6add7bc25e9491a888a140d254d293a985324a4d'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Mistral 13B Fine-Tune

# Imports

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [ ]:
!rm -rf /opt/conda/lib/python3.10/site-packages/aiohttp-3.9.1.dist-info
!pip install bitsandbytes
!pip install transformers
!pip install peft
!pip install accelerate
!pip install trl


  Using cached trl-0.8.6-py3-none-any.whl.metadata (11 kB)
  Using cached tyro-0.8.4-py3-none-any.whl.metadata (7.9 kB)
  Using cached shtab-1.7.1-py3-none-any.whl.metadata (7.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 4.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 6.8 MB/s eta 0:00:00


In [ ]:
import warnings
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from datasets import Dataset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import cohen_kappa_score
from tokenizers import AddedToken
warnings.simplefilter('ignore')
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import load_dataset
from trl import SFTTrainer

In [ ]:
class PATHS:
    train_path = 'TRAINING_DATA_FROM_COMPETITION'
    test_path = 'TEST_DATA_FROM_COMPETITION'
    sub_path = 'SUBMISSION_PATH_FOR_COMPETITION'
    base_model = "/kaggle/input/mistral/pytorch/7b-v0.1-hf/1"
    new_model = "/kaggle/output/models/UUSI_MALLI"

In [ ]:
class CFG:
    n_splits = 5
    seed = 42
    max_length = 1024
    lr = 2e-4
    train_batch_size = 4
    eval_batch_size = 8
    train_epochs = 4
    weight_decay = 0.001
    warmup_ratio = 0.03
    num_labels = 6

# Load


In [ ]:
data = pd.read_csv(PATHS.train_path)
data['label'] = data['score'].apply(lambda x: x-1)
data["label"] = data["label"].astype('float32')
data.head()

,essay_id,full_text,score,label
0,000d118,Many people have car where they live. The thin...,3,2.0
1,000fe60,I am a scientist at NASA that is discussing th...,3,2.0
2,001ab80,People always wish they had the same technolog...,4,3.0
3,001bdc0,"We all heard about Venus, the planet without a...",4,3.0
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3,2.0


# Set Training Args


In [ ]:
training_args = TrainingArguments(
    output_dir=f'output_v{VER}',
    learning_rate=CFG.lr,
    per_device_train_batch_size=CFG.train_batch_size,
    per_device_eval_batch_size=CFG.eval_batch_size,
    num_train_epochs=CFG.train_epochs,
    weight_decay=CFG.weight_decay,
    evaluation_strategy='epoch',
    metric_for_best_model='qwk',
    save_strategy='epoch',
    save_total_limit=1,
    load_best_model_at_end=True,
    report_to='none',
    warmup_ratio=CFG.warmup_ratio,
    lr_scheduler_type='constant',
    optim='paged_adamw_32bit',
    logging_first_step=True,
    gradient_accumulation_steps=1
)


# Training


In [ ]:
# Get train and val data
train = data
valid = data.copy()

# Load base model (Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

model = AutoModelForCausalLM.from_pretrained(
        PATHS.base_model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(PATHS.base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

# Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

dataset = Dataset.from_pandas(data)
    # Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="full_text",
    tokenizer=tokenizer,
    args=training_args,
    packing= False,
)
trainer.train()

# Save model and tokenizer
trainer.save_model(f'mistral-7b_fold_{fold}_v{VER}')
tokenizer.save_pretrained(f'mistral-7b_fold_{fold}_v{VER}')

trainer.model.save_pretrained(PATHS.new_model)

Map:   0%|          | 0/17307 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 